In [1]:
import pandas as pd
import plotly_express as px

file_path = "Data/Folkhalsomyndigheten_Covid19.xlsx"
sheet_name = "Totalt antal per åldersgrupp"

df_antal_fall_per_åldersgrupp = pd.read_excel(file_path, sheet_name = sheet_name)
df_antal_fall_per_åldersgrupp.rename(columns = {"Totalt_antal_fall": "Antal Fall", "Totalt_antal_intensivvårdade": "Antal Intensivvårdade", "Totalt_antal_avlidna": "Antal Avlidna"}, inplace = True)
# TODO regex out underscores in Ålder
df_antal_fall_per_åldersgrupp.head()

,Åldersgrupp,Antal Fall,Antal Intensivvårdade,Antal Avlidna
0,Ålder_0_9,138071,109,17
1,Ålder_10_19,355823,101,9
2,Ålder_20_29,418506,285,41
3,Ålder_30_39,493443,492,71
4,Ålder_40_49,474702,997,172


In [2]:
# plot showing how badly different age groups are affected by the virus
# TODO title and labels
px.histogram(df_antal_fall_per_åldersgrupp, x = "Åldersgrupp", y = ["Antal Fall", "Antal Intensivvårdade", "Antal Avlidna"], barmode = "group", log_y = True)